<a href="https://colab.research.google.com/github/pgallino/Organizacion-de-datos-TP1-1C2023/blob/main/pandas27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


27) ¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma? (⭐⭐)

HAY DOS RESOLUCIONES, LA PRIMERA LA "CORRECTA" TRABAJANDO SOBRE UN SOLO DATAFRAME, Y LA SEGUNDA DIVIDIENDO EL DATAFRAME EN DOS Y MERGEANDO

Ya analicé el csv y para no leer todo, solo cargo las columnas 'timestamp', 'contributor_username', 'logtype', 'action' y 'title'.

In [ ]:
df_logs = pd.read_csv('/content/drive/MyDrive/Wikipedia dataset 2c2021/logs.csv', usecols = [1,2,6,7,8])

In [ ]:
df_logs.head(10)

,timestamp,contributor_username,logtype,action,title
0,2004-12-23T07:10:09Z,Sabbut,delete,delete,Verificacionismo
1,2004-12-23T07:14:09Z,Sabbut,delete,delete,Rapunzel
2,2004-12-23T07:26:10Z,Sabbut,delete,delete,Macromedia Flash
3,2004-12-23T07:55:09Z,Sanbec,delete,delete,Archivo:Gnu head.png
4,2004-12-23T08:02:22Z,Sanbec,delete,delete,Archivo:Gnu-head-sm.jpg
5,2004-12-23T08:28:32Z,Dodo,block,block,Usuario:212.55.30.157
6,2004-12-23T12:10:06Z,Antoine,upload,upload,Archivo:La florida.jpg
7,2004-12-23T12:12:15Z,Antoine,upload,upload,Archivo:La florida uoct.jpg
8,2004-12-23T12:13:43Z,Antoine,upload,upload,Archivo:Pablo zalaquett.jpg
9,2004-12-23T12:23:54Z,Sanbec,delete,delete,Archivo:Szymborska znaczek.jpg


filtro 'action' por 'block' o 'logtype' por 'newusers'

In [ ]:
df_logs = df_logs.loc[(df_logs['logtype'] == 'newusers') | (df_logs['action'] == 'block')]

defino una función para limpiar el title, si action es 'newusers', no hace falta limpiar.

In [ ]:
def limpiar_title(x):
  if (x != 'Especial:Userlogin'):
    return x.split(":")[1]
  return x

In [ ]:
df_logs['title'] = df_logs.title.map(lambda x : limpiar_title(x), na_action = 'ignore')

Paso los contributor_username de los action 'newusers' a la columna title para tener el nombre de usuario de todos en una misma columna.

In [ ]:
df_logs.loc[df_logs['action'] == 'newusers', 'title'] = df_logs.contributor_username

Por errores del csv, saco los nan.

In [ ]:
df_logs = df_logs.loc[~(df_logs.title.isnull())]

Hay usuarios que fueron bloqueados pero no registrados. Además, hay usuarios que fueron registrados pero no bloqueados. Para quedarme con los que fueron bloqueados y registrados, cuento la cantidad de acciones que tuvo cada usuario. Si es 2, quiere decir que fue registrado y bloqueado, si no, es 1, bloqueado o registrado.

In [ ]:
df_logs['cant_acciones'] = df_logs.groupby('title')['action'].transform('nunique')

In [ ]:
df_logs = df_logs.loc[ 3 > df_logs['cant_acciones'] > 1]

Para trabajar con las fechas, las paso a tipo datetime.

In [ ]:
df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'])

Ahora, que ya tengo 2 filas por usuario, una por registro y otra por bloqueo, hago min y max sobre los timestamp, agrupando por usuario. Min es la fecha de registro, y max la fecha de bloqueo.

In [ ]:
df_total = df_logs.groupby('title').agg({"timestamp":["min","max"]})

Elimino los que por error del csv se registran posteriormente a ser bloqueados.

In [ ]:
df_total = df_total.loc[df_total['timestamp']['max'] > df_total['timestamp']['min']]

Calculo la diferencia entre las fechas y me quedo el mínimo.

In [ ]:
(df_total['timestamp']['max'] - df_total['timestamp']['min']).min()

Timedelta('0 days 00:00:04')

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

SEGUNDA RESOLUCIÓN, SEPARANDO EL DATAFRAME ENTRE BLOQUEADOS Y NEWUSERS, MERGEANDO DESPUÉS.

Para encontrar los bloqueados, filtro por "action" == "block", ya que solo me importan los de action block. En logtype "block" también está "unblock" y "reblock" pero no me interesan.

In [ ]:
df_blocks = df_logs.loc[(df_logs.action == 'block') & ~(df_logs.title.isnull())]
df_blocks["title"] = df_blocks.title.map(lambda x : x.split(":")[1]) #limpio el "usuario:"
df_blocks = df_blocks.rename(columns={'title':'usuario', 'timestamp': 'tiempo_bloqueo'})
df_blocks

<ipython-input-17-78be4fe2299f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_blocks["title"] = df_blocks.title.map(lambda x : x.split(":")[1]) #limpio el "usuario:"


,tiempo_bloqueo,contributor_username,logtype,action,usuario
5,2004-12-23 08:28:32+00:00,Dodo,block,block,212.55.30.157
287,2004-12-27 17:54:50+00:00,Dodo,block,block,201.241.228.15
288,2004-12-27 17:57:12+00:00,Dodo,block,block,201.255.50.32
290,2004-12-27 17:59:56+00:00,Dodo,block,block,84.122.0.158
313,2004-12-28 01:38:59+00:00,Comae,block,block,81.192.12.93
...,...,...,...,...,...
12188865,2021-09-03 02:05:46+00:00,Taichi,block,block,186.141.200.121
12188886,2021-09-03 02:24:41+00:00,Hasley,block,block,Galaxi066
12188909,2021-09-03 02:52:42+00:00,Marcelo,block,block,Xx.tkpro.xx
12188924,2021-09-03 03:00:14+00:00,Hasley,block,block,181.167.104.152


como los pueden bloquear más de una vez, me quedo con la primera vez

In [ ]:
df_blocks = df_blocks.groupby('usuario', as_index = False).agg({"tiempo_bloqueo":"min"})

Obtengo los usuarios registrados. Analicé el csv y en Logtype 'newusers' hay varias actions. Interpreto que son distintas formas de crear un nuevo usuario. Entre ellas, 'newusers', 'autocreate' y 'create' que tienen el nombre de usuario en 'contributor_username'. Por otro lado, 'create2' y 'byemail' tienen el nombre de usuario en 'title' de la forma 'usuario:<nombre_usuario>'

In [ ]:
df_newusers_cu = df_logs.loc[((df_logs.action == 'newusers') | (df_logs.action == 'autocreate') | (df_logs.action == 'create')) & ~(df_logs.contributor_username.isnull())]
df_newusers_cu = df_newusers_cu.rename(columns={'contributor_username':'usuario', 'timestamp': 'tiempo_registro'})
df_newusers_cu = df_newusers_cu[['tiempo_registro','usuario']]

df_newusers_ti = df_logs.loc[((df_logs.action == 'create2') | (df_logs.action == 'byemail')) & ~(df_logs.title.isnull())]
df_newusers_ti["title"] = df_newusers_ti.title.map(lambda x : x.split(":")[1])
df_newusers_ti = df_newusers_ti.rename(columns={'title':'usuario', 'timestamp': 'tiempo_registro'})
df_newusers_ti = df_newusers_ti[['tiempo_registro','usuario']]

<ipython-input-24-4df6d3a15edd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newusers_ti["title"] = df_newusers_ti.title.map(lambda x : x.split(":")[1])


In [ ]:
df_newusers_ti.head(10)

,tiempo_registro,usuario
202831,2006-04-19 16:46:58+00:00,Julioaguirre
208634,2006-04-24 18:10:23+00:00,LUIS RIVERA
211614,2006-04-27 00:16:23+00:00,Lisseth Mendoza
214892,2006-04-30 05:18:12+00:00,Antonio E
214894,2006-04-30 05:30:14+00:00,Berthard
214896,2006-04-30 05:34:05+00:00,Dorieo21~eswiki
214897,2006-04-30 05:34:06+00:00,Aydee~eswiki
215977,2006-05-01 04:40:14+00:00,Patuber
216629,2006-05-01 20:23:55+00:00,Jorge roberto lopez
216674,2006-05-01 20:48:50+00:00,Diego fernando~eswiki


In [ ]:
df_newusers_cu.head(10)

,tiempo_registro,usuario
45790,2005-09-07 22:33:56+00:00,Yliana
45801,2005-09-07 22:50:58+00:00,BrokenSegue
45806,2005-09-07 22:57:39+00:00,Jose luis pereira
45807,2005-09-07 22:59:55+00:00,Yatza~eswiki
45810,2005-09-07 23:00:47+00:00,JaN~eswiki
45813,2005-09-07 23:07:44+00:00,Edgar aleman
45814,2005-09-07 23:08:36+00:00,Ing. Enrique Hernandez Garcia
45815,2005-09-07 23:09:23+00:00,Ninucas
45819,2005-09-07 23:21:39+00:00,Nag~eswiki
45820,2005-09-07 23:24:32+00:00,David Julian


Una vez obtenidos todos los nuevos usuarios con sus fechas de registro, los uno en un solo dataframe.

In [ ]:
df_newusers = pd.concat([df_newusers_cu, df_newusers_ti]) #uno los dos tipos de nuevos_usuarios
df_newusers.head(10)

,tiempo_registro,usuario
45790,2005-09-07 22:33:56+00:00,Yliana
45801,2005-09-07 22:50:58+00:00,BrokenSegue
45806,2005-09-07 22:57:39+00:00,Jose luis pereira
45807,2005-09-07 22:59:55+00:00,Yatza~eswiki
45810,2005-09-07 23:00:47+00:00,JaN~eswiki
45813,2005-09-07 23:07:44+00:00,Edgar aleman
45814,2005-09-07 23:08:36+00:00,Ing. Enrique Hernandez Garcia
45815,2005-09-07 23:09:23+00:00,Ninucas
45819,2005-09-07 23:21:39+00:00,Nag~eswiki
45820,2005-09-07 23:24:32+00:00,David Julian


Con todos los newusers, y los bloqueados, armo un dataframe mergeandolos de forma que me queden todos los bloqueados, con su tiempo de bloqueo y tiempo de registro.

In [ ]:
df_total = pd.merge(df_blocks,df_newusers, how = 'inner')

In [ ]:
df_total.head(10) #son todos los bloqueados que se habian registrado, son menos que todos los bloqueados porque también bloquean gente sin usuario.

,tiempo_bloqueo,contributor_username,logtype,action,usuario,tiempo_registro
0,2005-05-11 09:59:48+00:00,Sanbec,block,block,Papa~eswiki,2016-03-23 16:20:36+00:00
1,2005-07-17 22:10:52+00:00,Angus,block,block,Carlos yo2,2006-02-08 15:20:40+00:00
2,2006-02-08 15:22:03+00:00,Alhen,block,block,Carlos yo2,2006-02-08 15:20:40+00:00
3,2005-09-09 13:38:54+00:00,Taragui,block,block,JohnGalt1812~eswiki,2005-09-09 02:00:11+00:00
4,2005-09-10 03:56:13+00:00,Taichi,block,block,-Piojoso-,2005-09-10 02:07:34+00:00
5,2005-09-11 19:37:38+00:00,Taichi,block,block,Di zeo,2005-09-10 18:02:03+00:00
6,2005-09-11 19:37:46+00:00,Taichi,block,block,Di zeo,2005-09-10 18:02:03+00:00
7,2005-09-11 19:53:08+00:00,Taichi,block,block,Di zeo,2005-09-10 18:02:03+00:00
8,2005-10-16 22:06:10+00:00,Taichi,block,block,Di zeo,2005-09-10 18:02:03+00:00
9,2006-01-15 07:12:55+00:00,Taichi,block,block,Di zeo,2005-09-10 18:02:03+00:00


Para trabajar con las fechas paso a datetime

In [ ]:
df_total['timestamp'] = pd.to_datetime(df_total['timestamp'])

Por un error del dataframe (supongo), hay usuarios que fueron bloqueados antes de registrarse, lo cual no tiene sentido. Asi que los descarto.

Puedo visualizar esos casos y son 32 en total.

In [ ]:
df_total.loc[df_total["tiempo_bloqueo"] < df_total["tiempo_registro"]]

,tiempo_bloqueo,contributor_username,logtype,action,usuario,tiempo_registro
0,2005-05-11 09:59:48+00:00,Sanbec,block,block,Papa~eswiki,2016-03-23 16:20:36+00:00
1,2005-07-17 22:10:52+00:00,Angus,block,block,Carlos yo2,2006-02-08 15:20:40+00:00
215,2006-02-02 18:50:24+00:00,Lourdes Cardenal,block,block,Vientodenieve~eswiki,2006-04-17 16:57:55+00:00
769,2006-06-14 19:40:54+00:00,Yrithinnd,block,block,Warko2006,2008-03-12 04:55:42+00:00
770,2007-07-21 20:03:05+00:00,Hispa,block,block,Warko2006,2008-03-12 04:55:42+00:00
820,2006-06-27 17:59:47+00:00,Hispa,block,block,Neutral~eswiki,2015-11-28 11:53:32+00:00
821,2006-06-28 18:19:37+00:00,Tostadora,block,block,Pathoschild,2007-04-06 05:51:30+00:00
916,2006-07-13 22:51:48+00:00,Petronas,block,block,SimeónES,2015-04-29 09:48:04+00:00
917,2006-12-17 22:48:53+00:00,Yrithinnd,block,block,SimeónES,2015-04-29 09:48:04+00:00
918,2007-02-27 00:39:18+00:00,Antur,block,block,SimeónES,2015-04-29 09:48:04+00:00


In [ ]:
df_total = df_total.loc[df_total["tiempo_bloqueo"] > df_total["tiempo_registro"]]

Finalmente, restando a la fecha de bloqueo, la fecha de registro, obtengo el tiempo entre el registro del usuario y su bloqueo. De todos ellos, me quedo con el mínimo

In [ ]:
resultado = (df_total['tiempo_bloqueo'] - df_total['tiempo_registro']).min()

In [ ]:
resultado

Timedelta('0 days 00:00:04')

El tiempo mínimo que ha durado desde su registro un usuario bloqueado en la plataforma, es de 4 segundos.